# TP4 

## Amaury BURTIN 

## Imports 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
import statsmodels as sm
import scipy
import powerlaw
import scipy.optimize as sc
import pandas as pd

## Part 1+2 : Implement Franke and Westerhoff model and Explore FW's model's behaviour.

Question 1

In [ ]:
#Parameters
NIT = 10000
R0 = 0
R1 = 0
X0 = 0
phi = 0.18
khi = 2.35
pstar = 0
mu = 0.01
nu = 2.57
alpha0 = -0.15
alphax = 1.35
alphad = 11.4
stdf = 0.79
stdc = 1.91

#Matrix initialization
R = np.zeros(NIT)
R[0] = R0
R[1] = R1
X = np.zeros(NIT)
X[0] = X0
TIME = np.arange(NIT)

In [ ]:
def P(R,t) :
    """Returns the price P[t]"""
    return(np.cumsum(R[:t])[t-1])

In [ ]:
def S(R,X,alpha0,alphax,alphad,pstar,t) :
    """Returns the choice of FW S[t]"""
    return(alpha0 + alphax*X[t] + alphad*(pstar-P(R,t))**2)

In [ ]:
def Normal(X,stdf,stdc,t) :
    """Returns E[t]"""
    std = np.sqrt(1/2*(((1+X[t])*stdf)**2 + ((1-X[t])*stdc)**2))
    return(np.random.normal(0,std))

In [ ]:
def Xup(R,X,nu,alpha0,alphax,alphad,pstar,t) :
    """Returns the next change in fraction chartists and fundamentalists X[t+1]"""
    St = S(R,X,alpha0,alphax,alphad,pstar,t)
    val = X[t]
    val += nu*((1-X[t])*np.exp(St)-(1+X[t])*np.exp(-St))
    if val > 1 :
        return(1)
    elif val < -1 :
        return(-1)
    else :
        return(val)

In [ ]:
def Rup(R,X,mu,phi,khi,pstar,stdf,stdc,t) :
    """Returns the next return R[t+1]"""
    val = mu/2
    fundamentalists = (1+X[t])*phi*(pstar-P(R,t))
    chartists = (1-X[t])*khi*(P(R,t)-P(R,t-1))
    epsilon = Normal(X,stdf,stdc,t)
    val*=(fundamentalists+chartists+epsilon)
    return(val)

In [ ]:
def evolution(X,R,NIT,nu,mu,phi,khi,pstar,stdf,stdc,alpha0,alphax,alphad) :
    """Let the return and the fraction of the two groups evolve over time"""
    for t in range(2,NIT-2) :
        X[t+1] = Xup(R,X,nu,alpha0,alphax,alphad,pstar,t)
        R[t+1] = Rup(R,X,mu,phi,khi,pstar,stdf,stdc,t)

In [ ]:
evolution(X,R,NIT,nu,mu,phi,khi,pstar,stdf,stdc,alpha0,alphax,alphad)

In [ ]:
plt.plot(TIME,np.cumsum(R))
plt.xlabel("Time")
plt.ylabel("Price")
plt.title("Price over time with two groups of agents")
plt.plot()

Question 2

In [ ]:
#Range for plotting
a = -2
b = -1.5
n = 1000

#Plotting sequences
X = np.logspace(a,b,n)
Y = np.zeros(n)

for i in range(n) :
    Y[i] = 1-ECDF(np.abs(R))(X[i])

In [ ]:
plt.plot(X,Y)
plt.xscale("log")
plt.yscale("log")
plt.title("1-ECDF of the return in logspace")
plt.show()

Question 3

In [ ]:
# Only evaluating the tail exponent on the 40 last percent of R (that are the most significant).
low_bound = int(len(R) - 4*len(R)/10)
powerlaw.Fit(np.abs(R[low_bound:])).power_law.alpha

This result comes from the scipy library, so we have to add +1 on the result. Hence the tail coefficient is 4.7

Question 4

In [ ]:
#Parameters initialization
NIT = 1000
R0 = 0
R1 = 0
X0 = 0
phi = 0.18
khi = 2.35
pstar = 0
mu = 0.01
nu = 2.57
alpha0 = -0.15
alphax = 1.35
alphad = 11.4
stdf = 0.79
stdc = 1.91

#Matrices initialization
R = np.zeros(NIT)
R[0] = R0
R[1] = R1
X = np.zeros(NIT)
X[0] = X0
TIME = np.arange(NIT)

#Parameter Iteration Number for plotting the tail exponent with respect to different variables. 
PIN = 30
Input = np.arange(PIN)



In [ ]:
#Plot initialization :
fig = plt.figure()
fig.set_size_inches(15, 10, forward=True)
plt.title("Dependance of the tail exponent on different parameters")

# Define low_bound based on current NIT
low_bound_temp = int(NIT - 4*NIT/10)

#Graph with respect to phi (0.18)
PHI = np.linspace(0.1,20,PIN)
TAIL_EXP = np.zeros(PIN)
for k in range(PIN) :
    # Reset arrays for each iteration
    R_temp = np.zeros(NIT)
    R_temp[0] = R0
    R_temp[1] = R1
    X_temp = np.zeros(NIT)
    X_temp[0] = X0
    evolution(X_temp,R_temp,NIT,nu,mu,PHI[k],khi,pstar,stdf,stdc,alpha0,alphax,alphad)
    # Use absolute values and filter out zeros to avoid powerlaw fitting issues
    abs_returns = np.abs(R_temp[low_bound_temp:])
    abs_returns = abs_returns[abs_returns > 1e-10]  # Remove very small values
    if len(abs_returns) > 10:  # Ensure enough data points
        TAIL_EXP[k] = powerlaw.Fit(abs_returns, verbose=False).power_law.alpha
    else:
        TAIL_EXP[k] = np.nan
plt.subplot(311)
plt.plot(PHI,TAIL_EXP,"r")
plt.ylabel("Tail exponent")
plt.title("Dependance on phi")

#Graph with respect to khi (2.35)
KHI = np.linspace(0.1,100,PIN)
TAIL_EXP = np.zeros(PIN)
for k in range(PIN) :
    # Reset arrays for each iteration
    R_temp = np.zeros(NIT)
    R_temp[0] = R0
    R_temp[1] = R1
    X_temp = np.zeros(NIT)
    X_temp[0] = X0
    evolution(X_temp,R_temp,NIT,nu,mu,phi,KHI[k],pstar,stdf,stdc,alpha0,alphax,alphad)
    abs_returns = np.abs(R_temp[low_bound_temp:])
    abs_returns = abs_returns[abs_returns > 1e-10]
    if len(abs_returns) > 10:
        TAIL_EXP[k] = powerlaw.Fit(abs_returns, verbose=False).power_law.alpha
    else:
        TAIL_EXP[k] = np.nan
plt.subplot(312)
plt.plot(KHI,TAIL_EXP,"b")
plt.ylabel("Tail exponent")
plt.title("Dependance on khi")

#Graph with respect to alpha0 (-0.15)
ALPHA0 = np.linspace(-5,5,PIN)
TAIL_EXP = np.zeros(PIN)
for k in range(PIN) :
    # Reset arrays for each iteration
    R_temp = np.zeros(NIT)
    R_temp[0] = R0
    R_temp[1] = R1
    X_temp = np.zeros(NIT)
    X_temp[0] = X0
    evolution(X_temp,R_temp,NIT,nu,mu,phi,khi,pstar,stdf,stdc,ALPHA0[k],alphax,alphad)
    abs_returns = np.abs(R_temp[low_bound_temp:])
    abs_returns = abs_returns[abs_returns > 1e-10]
    if len(abs_returns) > 10:
        TAIL_EXP[k] = powerlaw.Fit(abs_returns, verbose=False).power_law.alpha
    else:
        TAIL_EXP[k] = np.nan
plt.subplot(313)
plt.plot(ALPHA0,TAIL_EXP,"y")
plt.ylabel("Tail exponent")
plt.title("Dependance on alpha0")

plt.tight_layout()
plt.show()

The number of iteration is quite low in order to reduce computation time. One can clearly see that there cluster of values that tend to maximize the tail exponent. This is the case for each variable. 

## Part 3 : Calibrate the model

Question 1

In [ ]:
# Read the data from the file
file_path = 'datatata.csv'  # Update this with the correct file path
df = pd.read_csv(file_path)

#Parameters of the plotting
a = -3
b = -1
n = 1000

#Sequences to plot
X = np.logspace(a,b,n)
Y = np.zeros(n)
for i in range(n) :
    Y[i] = 1-ECDF(np.abs(df["0"]))(X[i])

plt.plot(X,Y)
plt.xlabel("input")
plt.ylabel("Return")
plt.yscale("log")
plt.title("1-ECDF of the studied series")
plt.show()


In [ ]:
#Fitting the tail exponent
powerlaw.Fit(np.abs(df["0"])).power_law.alpha

Question 2.1

In [ ]:
#Redefining the evolution function to be phi, khi and alpha0 dependant. 
def evolution2(phi,khi,alpha0) :
    phi = phi**2
    khi = khi**2
    NIT = 100
    R0 = 0
    R1 = 0
    X0 = 0
    pstar = 0
    mu = 0.01
    nu = 2.57
    alphax = 1.35
    alphad = 11.4
    stdf = 0.79
    stdc = 1.91
    R = np.zeros(NIT)
    R[0] = R0
    R[1] = R1
    X = np.zeros(NIT)
    X[0] = X0
    TIME = np.arange(NIT)
    for t in range(2,NIT-2) :
        X[t+1] = Xup(R,X,nu,alpha0,alphax,alphad,pstar,t)
        R[t+1] = Rup(R,X,mu,phi,khi,pstar,stdf,stdc,t)
    return(R)

R = evolution2(phi=np.sqrt(0.18),khi=np.sqrt(2.35),alpha0=-0.15)

In [ ]:
#Function that computes the tail exponent.
def func(phi,khi,alpha0) :
    R = evolution2(phi,khi,alpha0)
    tail_exp = powerlaw.Fit(np.abs(R), verbose=False).power_law.alpha
    return(tail_exp)

def loss(X) :
    return((9.2 - func(X[0],X[1],X[2])**2))

sc.minimize(loss,np.array([np.sqrt(0.18),np.sqrt(2.35),-0.15]))

One obtains a vector x of the parameters to plug to minimize the loss. Lets see if they do what they should through the func function, which outputs the tail exponent of a series generated with different parameters. 

In [ ]:
x =  [ 4.243e-01, 1.533e+00, -1.500e-01]
func(x[0],x[1],x[2])

The simulation does not seem near reality, even after having replaced the positive parameters with their square root and then their square. Lets try to rerun the minimization operation but with another starting point (0,0,0) for instance. 

In [ ]:
sc.minimize(loss,np.array([0,0,0]))

Here the minimization parameters are really different from the first minimization. 

In [ ]:
x =  [ 1.825e-06, -1.096e-07, 6.967e-07]
func(x[0],x[1],x[2])

The result is once more not very close to reality. The difference could come from the starting point. Indeed, with 3 degrees of freedom, the shape of the function linking phi, khi and alpha0 is very complex and can hide many minima. 

Question 2.2

Lets try manually by first acting on the parameters that have the most impact on the tail exponent (trying values close to maximizing the tail exponent). The goal of the following code is to find the set of parameters that will make the ouput of func as close as 9.2 as possible. The function is func(phi,khi,alpha0).

In [ ]:
print(func(0,0,0))
print(func(4,0,0))
print(func(0,26,0))
print(func(0,0,1.5))
print(func(0,10,1.5))
print(func(2,10,1.5))
print(func(3,10,1.5))

In [ ]:
print(func(3,26,1.5))
print(func(4,26,1.5))
print(func(4,24,1.5))

The last parameters seem to fit the best, even though there are a lot of variations due to the gaussian noise in the process. Let's use these variables.

## Part 3 : Calibrate II

Question 1 :

In [ ]:
#Calculation of tail exponent many times
X = [4,24,1.5]
NIT = 20
C = np.zeros(NIT)

for i in range(NIT) :
    C[i]=func(X[0],X[1],X[2])

Mean = np.mean(C)
print(Mean)

The mean (4.97) is high, but still not close to cfill = 9.2.

In [ ]:
std = np.std(C)
print(std)

The std is quite high for such application, and the 99% interval does not even include the target value. This means that the noise applied to the return makes it very unpredictable. 

On the whole, trying to predict the price evolution or heavy tails event in agent based model is a tricky question, and requires very complex models that can deal with the latter uncertainty. 